In [3]:
import pandas as pd
import numpy as np
import requests
import gzip
import re

In [4]:
file = pd.read_csv(gzip.open('data/gun_violence_massshooting_2014-2018.csv.gz'))                       # 239677 Rows, 30 Columns
file1 = file[['incident_id', 'date', 'state', 'city_or_county', 'address', 'n_killed',                 # 239677 Rows, 11 Columns
              'n_injured', 'sum n_killed&n_injured', 'latitude', 'longitude', 'n_guns_involved']]
file2 = file[['incident_id', 'gun_type', 'incident_characteristics', 'participant_age',                # 239677 Rows, 9 Columns
              'participant_age_group', 'participant_gender','participant_name', 'participant_status',
              'participant_type']]
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
states_dict = {value: key for key, value in states.items()}
file1['STATE'] = file1['state'].replace(states_dict)

C:\Users\zhua1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
# Do a dropna on file2, entire file is too big for analysis
file2 = file2.dropna(axis = 0).reset_index(drop = True)
def countVictims(dt):
     return dt.count('Victim')
def countSuspects(dt):
     return dt.count('Subject-Suspect')
file2['# of Victims'] = file2['participant_type'].apply(countVictims)
file2['# of Suspects'] = file2['participant_type'].apply(countSuspects)

In [6]:
# # of Male Victims is NAN when the dataset does not have the gender for each victim. len(gender) != len(victim)
def Vict(typ, gen, phrase):
    male_female_vict = {}
    male_victim = []
    female_victim = []
    part_type = re.sub('\d::', '', typ).split('||')
    part_gender = re.sub('\d::', '', gen).split('||')
    # Get number of victims that are Male
    # Find index of victims in part_type list
    if phrase in part_type:
        idx = [i for i, x in enumerate(part_type) if x == phrase]
        if (len(part_type) != len(part_gender)):
            pass
        else:
            if 'Male' in np.array(part_gender)[idx] and 'Female' in np.array(part_gender)[idx]:
                male_victim.append(list(np.array(part_gender)[idx]).count('Male'))
                female_victim.append(list(np.array(part_gender)[idx]).count('Female'))
            elif 'Male' in np.array(part_gender)[idx]:
                male_victim.append(list(np.array(part_gender)[idx]).count('Male'))
                female_victim.append(0)
            elif 'Female' in np.array(part_gender)[idx]:
                female_victim.append(list(np.array(part_gender)[idx]).count('Female'))
                male_victim.append(0)
        if len(male_victim) != 1:
            male_victim = [np.nan]
        if len(female_victim) != 1:
            female_victim = [np.nan]
        male_female_vict['male'] = male_victim[0]
        male_female_vict['female'] = female_victim[0]
    else:
        male_female_vict['male'] = np.nan
        male_female_vict['female'] = np.nan
    return male_female_vict

In [7]:
file2['# of Male Victims'] = file2.apply(lambda x: Vict(x['participant_type'], x['participant_gender'], 'Victim')['male'], axis=1)
file2['# of Female Victims'] = file2.apply(lambda x: Vict(x['participant_type'], x['participant_gender'], 'Victim')['female'], axis=1)
file2['# of Male Suspects'] = file2.apply(lambda x: Vict(x['participant_type'], x['participant_gender'], 'Subject-Suspect')['male'], axis=1)
file2['# of Female Suspects'] = file2.apply(lambda x: Vict(x['participant_type'], x['participant_gender'], 'Subject-Suspect')['female'], axis=1)

In [8]:
# Inner merge with file1
final = file1.merge(file2, how = 'inner', on = 'incident_id')
# Convert date column from Obejct to Date
final['date'] =  pd.to_datetime(final['date'], format='%m/%d/%Y')
final['year'] = pd.DatetimeIndex(final['date']).year
# Delete year 2013
final_df = final[final['year'] != 2013]
###################################################### 69265 Rows, 27 Columns ##################################################

In [10]:
final_df.to_csv('C:/Users/zhua1/Desktop/cleanedBigGun_gender.csv', index = False)